In [1]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [3]:
model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')

In [4]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [6]:
features = extract_features('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/image_0013.jpg', model)
print(len(features))

1/1 [==============================] - 0s 426ms/step
2048


In [7]:
%timeit features = extract_features('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/image_0013.jpg', model)

1/1 [==============================] - 0s 49ms/step
62.7 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [10]:
# path to the your datasets
root_dir = '/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/caltech-101'
filenames = sorted(get_file_list(root_dir))

feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

/var/folders/z4/vgf_bzqj0s74161f8l8m12_h0000gn/T/ipykernel_8592/3568103458.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(filenames))):


  0%|          | 0/8677 [00:00<?, ?it/s]

1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 49ms/step


...

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 44ms/step


In [11]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict_generator(generator, num_epochs)
end_time = time.time()

Found 8677 images belonging to 2 classes.


/var/folders/z4/vgf_bzqj0s74161f8l8m12_h0000gn/T/ipykernel_8592/1637432355.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  feature_list = model.predict_generator(generator, num_epochs)
2023-05-27 01:06:13.176966: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [12]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 2048)
Time taken in sec =  242.00273609161377


In [13]:
filenames = [root_dir + '/' + s for s in generator.filenames]

In [15]:
pickle.dump(generator.classes, open('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/class_ids-caltech101.pickle',
                                    'wb'))
pickle.dump(filenames, open('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/filenames-caltech101.pickle', 'wb'))
pickle.dump(
    feature_list,
    open('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/features-caltech101-' + model_architecture + '.pickle', 'wb'))

In [23]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [17]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)

In [18]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')

Found 8677 images belonging to 2 classes.


In [19]:
def model_maker():
    base_model = ResNet50(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [24]:
model_finetuned = model_maker()
model_finetuned.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model_finetuned.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs=10)

/var/folders/z4/vgf_bzqj0s74161f8l8m12_h0000gn/T/ipykernel_8592/4229963176.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_finetuned.fit_generator(


Epoch 1/10


2023-05-27 01:23:19.687888: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


136/136 [==============================] - 254s 2s/step - loss: 0.0124 - acc: 0.9949
Epoch 2/10
136/136 [==============================] - 276s 2s/step - loss: 2.4283e-06 - acc: 1.0000
Epoch 3/10
136/136 [==============================] - 286s 2s/step - loss: 1.5714e-06 - acc: 1.0000
Epoch 4/10
136/136 [==============================] - 290s 2s/step - loss: 1.6311e-06 - acc: 1.0000
Epoch 5/10
136/136 [==============================] - 289s 2s/step - loss: 1.0333e-06 - acc: 1.0000
Epoch 6/10
136/136 [==============================] - 282s 2s/step - loss: 3.2983e-06 - acc: 1.0000
Epoch 7/10
136/136 [==============================] - 260s 2s/step - loss: 6.6587e-07 - acc: 1.0000
Epoch 8/10
136/136 [==============================] - 261s 2s/step - loss: 5.7998e-07 - acc: 1.0000
Epoch 9/10
136/136 [==============================] - 263s 2s/step - loss: 1.1774e-06 - acc: 1.0000
Epoch 10/10
136/136 [==============================] - 258s 2s/step - loss: 1.1249e-06 - acc: 1.0000


In [25]:
model_finetuned.save('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/model-finetuned.h5')

/Users/rushabhwalke/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
start_time = time.time()
feature_list_finetuned = []
feature_list_finetuned = model_finetuned.predict_generator(generator, num_epochs)
end_time = time.time()

for i, features_finetuned in enumerate(feature_list_finetuned):
    feature_list_finetuned[i] = features_finetuned / norm(features_finetuned)

feature_list = feature_list_finetuned.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

/var/folders/z4/vgf_bzqj0s74161f8l8m12_h0000gn/T/ipykernel_8592/4162386107.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  feature_list_finetuned = model_finetuned.predict_generator(generator, num_epochs)
2023-05-27 02:11:03.486365: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Num images   =  8677
Shape of feature_list =  (8677, 2)
Time taken in sec =  241.04401993751526


In [27]:
pickle.dump(
    feature_list,
    open('/Users/rushabhwalke/Desktop/CV Projects/reverse-image-search-engine/features-caltech101-resnet-finetuned.pickle', 'wb'))